## Homework 2 (Part 1, 2, and 3) - Roshan Poudel

In [1]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

2023-02-10 00:01:33.879662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_images.shape

(60000, 28, 28)

> The shape is the tuple of integers that describe how many dimensions the tensor has along each axis<br>The output here means that train_images is a 3-D tensor with a collection of 60000, 28 by 28 matrices

In [3]:
len(train_images.shape)

3

> len(train_images.shape) equivalent to calling train_images.ndim below

In [4]:
train_images.ndim

3

In [5]:
my_slice = train_images[10:100]
my_slice.shape

(90, 28, 28)

In [6]:
my_slice = train_images[10:100, :, :]
my_slice.shape

(90, 28, 28)

In [7]:
my_slice = train_images[10:100, 0:28, 0:28]
my_slice.shape

(90, 28, 28)

**In order to select 14 x 14 pixels in the bottom-right corner of *all* images, we would do the following:**

In [8]:
my_slice = train_images[:, 14:, 14:]
my_slice.shape

(60000, 14, 14)

In [9]:
batch = train_images[:128]

> Up here is the First batch of training data

In [10]:
batch = train_images[128:256]

> Second batch of training data <br>

> **nth batch would be:**<br>
>```py
n = 3
batch = train_images[128 * n: 128 * (n+1)]
```

### Naive python implementation of element-wise relu operation:

In [11]:
def naive_relu(x):
    assert len(x.shape) == 2 #making sure the array is 2D
    x = x.copy() # we don't want to make changes to original array
    for i in range(x.shape[0]): #x.shape[0] gives the length of the whole array cuz we need to iterate through all elements
        for j in range(x.shape[1]): #x.shape[1] gives the length of the individual arrays in the array as we need to loop through every col
            x[i, j] = max(x[i,j], 0) #relu operation: relu(x) is max(x, 0)
    return x

In [12]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] += y[i,j] #adding every element in y to the corresponding element in x
    return x

> In NumPy, we can do the following element-wise operation and it will be blazing fast:
```py
import numpy as np
z = x + y #Element-wise addition
z = np.maximum(z, 0.) #Element-wise relu
```
**Let's try**

In [13]:
import time
import numpy as np

x = np.random.random((20, 100))
y = np.random.random((20,100))

t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.)
print("Took:{0:.2f} s". format(time.time() - t0))

Took:0.01 s


> Using numpy took 0.01. Let's try with the naive version

In [14]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("Took:{0:.2f} s". format(time.time() - t0))

Took:1.89 s


> It took 1.80s with the naive approach. That means that the numpy implementation is about 180 times faster than the naive implementation

### The following example shows what broadcasting is

In [15]:
import numpy as np
#for the sake of simplicity, I used a tensor of shape (3, 4) instead of the one in the book (32, 10)
X = np.random.random((3,4))
print("X:\n", X, "\n")
y = np.random.random((4,))
print("y:\n", y, "\n")
y = np.expand_dims(y, axis=0)
print("y with shape (1,4):\n", y, "\n")
Y = np.concatenate([y] * 3, axis=0)
print("Y with shape same as X:\n", Y)
#After this we can simply add X and Y as they have the same shape

X:
 [[0.75979652 0.39508733 0.92433174 0.15084463]
 [0.14185885 0.17025137 0.06997549 0.48750564]
 [0.72520224 0.66928034 0.92039891 0.57594182]] 

y:
 [0.6211849  0.17038381 0.91008597 0.21820181] 

y with shape (1,4):
 [[0.6211849  0.17038381 0.91008597 0.21820181]] 

Y with shape same as X:
 [[0.6211849  0.17038381 0.91008597 0.21820181]
 [0.6211849  0.17038381 0.91008597 0.21820181]
 [0.6211849  0.17038381 0.91008597 0.21820181]]


In [16]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

## Tensor Product

[]_mxn * []nxp = []_mxp <br>

**For two matrices to be multiplicable, the number of columns in the first matrix must equal to the number of rows in the second matrix**

Eg: a 2x3 matrix can be multiplied with a 3*4 matrix. It will result in a matrix of size 2X4.

In [17]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    z = 0
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

In [18]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

> Same code can be implemented as
>>```py
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z
```


In [19]:
# For the computation of the dot product of matrices, we can use the following implementation
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]): 
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

**Testing the time taken for the computation of the dot product using naive vs using numpy library**

In [25]:
Mat_1 = np.random.random((20, 100))
Mat_2 = np.random.random((100,30))
print(type(Mat_1))

#USING NUMPY
t_initial = time.time()
for _ in range(500):
    dot_product = np.dot(Mat_1, Mat_2)
print("Took:{0:.2f} s with numpy". format(time.time() - t_initial))

#Using our naive approach for dot product computation
t_initial = time.time()
for _ in range(500):
    dot_product = naive_matrix_dot(Mat_1, Mat_2)
print("Took:{0:.2f} s with naive_matrix_product". format(time.time() - t_initial))

<class 'numpy.ndarray'>
Took:0.02 s with numpy
Took:12.08 s with naive_matrix_product


>*Looks like our naive guy is significantly slower than blazing fast BLAS implemented NumPy function .dot()*

## Tensor Reshaping

>Reshaping a tensor means rearranging its rows and columns to match a target shape. Naturally, the reshaped tensor has the same total number of coefficients as the initial tensor.

In [21]:
x = np.array([[0., 1.],
             [2.,3.],
             [4.,5.]])
x.shape

(3, 2)

In [22]:
x = x.reshape((6,1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [23]:
x = x.reshape((2,3))
x

array([[0., 1., 2.],
       [3., 4., 5.]])

> **Transpose of a m X n matrix simply means that the rows and columns are exchanged to form the same matrix as n X m dimensions.** 
Here is an example:

In [24]:
x = np.zeros((300, 20)) #creating an all zeros matrix of shape(300, 20)
x = np.transpose(x)
x.shape

(20, 300)